In [3]:
import os
import json
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [4]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

716789
705921


In [5]:
df = df.filter(pl.col("stran") >= 30)

In [6]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))
cesi = aut.explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(pl.col("100_7")).to_series().to_list()
len(cesi)

364420

In [7]:
vek = pl.read_parquet(os.path.join("data","narozeni-umrti-gender.parquet"))
df = df.join(vek, on="100_7", how="left").with_columns((pl.col("rok") - pl.col("narozeni")).alias("vek"))

In [8]:
df_700 = pl.read_parquet(os.path.join("data","podil_lidi_s_pauzou.parquet")).with_columns(pl.col("rok").map_elements(lambda x: x.year)).filter(pl.col("rok").is_between(1910,1990))

In [9]:
alt.Chart(alt_friendly(df_700)).mark_line().encode(alt.X("rok:T"), alt.Y("podil_cechu_ill_10"))

alt.Chart(...)

In [10]:
alt.Chart(alt_friendly(df_700)).mark_line().encode(alt.X("rok:T"), alt.Y("podil_cechu_trl_10"))

alt.Chart(...)

## Autorstvo s pauzou a končící autorstvo

In [12]:
df_100 = pl.read_parquet(os.path.join("data","podil_autorstva_s_pauzou.parquet")).with_columns(pl.col("rok").map_elements(lambda x: x.year)).filter(pl.col("rok").is_between(1900,2000)).with_columns(pl.col("podil_cechu_15").map_elements(lambda x: round(x, 3)))
df_koncici = pl.read_parquet(os.path.join('data','podily_konciciho_autorstva_cs.parquet'))

### Záhada roku 1905

In [14]:
df = df.with_columns(pl.col("100_a").map_elements(hezke_jmeno).alias("jmeno"))

In [15]:
rok04 = df.filter(pl.col("rok") == 1904).select(pl.col("100_7")).to_series().to_list()
roky06az18 = df.filter(pl.col("rok").is_between(1906,1918)).select(pl.col("100_7")).to_series().to_list()
roky18plus = df.filter(pl.col("rok") > 1900).select(pl.col("100_7")).to_series().to_list()
odmlceni_05 = [x for x in rok04 if x in roky18plus and x not in roky06az18]

In [16]:
df.filter(pl.col('100_7').is_in(odmlceni_05)).group_by(['jmeno','100_7']).len().sort(by='len',descending=True).head(20)

jmeno,100_7,len
str,str,u32
"""Ladislav Klíma""","""jk01060531""",75
"""Antonín Gindely""","""jk01032434""",50
"""Václav Vladivoj Tomek""","""jk01132708""",40
"""František Cajthaml-Liberté""","""jk01020537""",25
"""František Vlasák""","""jk01150047""",24
"""Martin von Cochem""","""jo2002109710""",22
"""Adolf Eckert""","""jk01030294""",22
"""František Cyril Kampelík""","""jk01052820""",21
"""Quintus Flaccus Horatius""","""jn20000603003""",19


In [17]:
', '.join(df.filter(pl.col('100_7').is_in(odmlceni_05)).group_by(['jmeno','100_7']).len().sort(by='len',descending=True).select(pl.col('jmeno')).to_series().to_list()[0:30])

'Ladislav Klíma, Antonín Gindely, Václav Vladivoj Tomek, František Cajthaml-Liberté, František Vlasák, Adolf Eckert, Martin von Cochem, František Cyril Kampelík, Quintus Flaccus Horatius, Hendrik Conscience, Antonín Konstantin Viták, Jan Karník, Jan Šimsa, Matyáš Blažek, Luboš Jeřábek, František Karel Bakule, Eugen Kumičić, František Ladislav Menhard, Leopold M. Zeithammer, Josef Šmaha, Václav Vondrák, František Buben, Jan Sommer, Václav Franěk, Martin Kuchynka, Robert Saudek, Rudolf Knaus, Josef Barák, E. V. Řičák, Eduard Georgievič Valečka'

In [18]:
graf_pauzy = alt.Chart(
    alt_friendly(df_100.filter(pl.col('rok').is_between(1930,1995))), width=kredity['sirka'], height=kredity['vyska_nizkych'] * 1.5,
    title=alt.Title("Podíl českých autorů, kteří se odmlčeli na 10 a více let")
).mark_bar(color='#E09DA3', width=3,
).encode(alt.X(
    "rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')
), alt.Y(
    "podil_cechu_10:Q", title=None, 
    axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', labelExpr="datum.label * 100 + ' %'", orient='right', tickCount=3),
    scale=alt.Scale(domain=[0,0.095])
))
                    
rule1 = alt.Chart(alt.Data(values=[{'rok': '1948-02-25'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

rule2 = alt.Chart(alt.Data(values=[{'rok': '1968-08-21'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

# Add text annotation
text1 = alt.Chart(alt.Data(values=[{'rok': '1948-02-25', 'y': 0.13}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    dy=5,# Slight horizontal offset from the line
    dx=-4,
    fontSize = 8,
    text=['komunistický','převrat'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)


# Add text annotation
text2 = alt.Chart(alt.Data(values=[{'rok': '1965-08-21', 'y': 1}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    dy=5,# Slight horizontal offset from the line
    dx=12,
    fontSize=8,
    text=['srpnová','okupace'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)

graf_pauzy_graf_komb = graf_pauzy + rule1 + text1 + rule2 + text2
graf_pauzy_graf_komb

alt.LayerChart(...)

In [19]:
graf_konce = alt.Chart(
    alt_friendly(df_koncici.filter(pl.col("rok").is_between(1900,2000))
                ), width=kredity['sirka'], height=kredity['vyska_nizkych'] * 1.5,
                 title=alt.Title(["Podíl českých autorů, kteří vydali svoji poslední knihu"])
).mark_bar(color='#E09DA3',width=3
).encode(
    alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    alt.Y("podil:Q", title=None, scale=alt.Scale(domain=[0,0.25]), axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', tickCount=3, labelExpr="datum.label * 100 + ' %'", orient='right')))
graf_konce

rule1 = alt.Chart(alt.Data(values=[{'rok': '1948-02-25'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

rule2 = alt.Chart(alt.Data(values=[{'rok': '1989-11-17'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

# Add text annotation
text1 = alt.Chart(alt.Data(values=[{'rok': '1952-02-25', 'y': 0.2}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='left',
    baseline='middle',
    dy=2,# Slight horizontal offset from the line
    dx=-9,
    fontSize = 8,
    text=['komunistický','převrat'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)


# Add text annotation
text2 = alt.Chart(alt.Data(values=[{'rok': '1987-11-17', 'y': 0.2}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    fontSize=8,
    dy=2,# Slight horizontal offset from the line
    dx=-2,
    text=['sametová','revoluce'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)

graf_konce_komb = graf_konce + text1 + text2

graf_konce_komb

alt.LayerChart(...)

In [20]:
pauzy_konce_final = alt.vconcat(
    graf_konce_komb, graf_pauzy_graf_komb
).configure_view(
    stroke='transparent'
).resolve_scale(
    y='shared'
).configure_axisX(
    grid=False, domain=False
)

pauzy_konce_final

alt.VConcatChart(...)

In [21]:
konce_final = graf_konce_komb.configure_view(
    stroke='transparent'
).resolve_scale(
    y='shared'
).configure_axisX(
    grid=False, domain=False
)

konce_final

alt.LayerChart(...)

In [22]:
pauzy_final = graf_pauzy_graf_komb.configure_view(
    stroke='transparent'
).resolve_scale(
    y='shared'
).configure_axisX(
    grid=False, domain=False
)

pauzy_final

alt.LayerChart(...)

In [23]:
me_to_neurazi(konce_final, soubor="04_konce", kredity=kredity['default'])

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/04_konce.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/04_konce.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [24]:
me_to_neurazi(pauzy_final, soubor="04_pauzy", kredity=kredity['default'])

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/04_pauzy.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/04_pauzy.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [25]:
# me_to_neurazi(pauzy_konce_final, soubor="04_konce_pauzy", kredity=kredity['default'])

## Kdo zmizel

In [27]:
pred_unorem = set(df.filter(pl.col("rok") <= 1948).select(pl.col("100_7")).to_series().to_list())
po_revoluci = set(df.filter(pl.col("rok") >= 1988).select(pl.col("100_7")).to_series().to_list())
v_sedesatkach = set(df.filter(pl.col("rok").is_between(1960,1969)).select(pl.col("100_7")).to_series().to_list())
behem_socialismu = set(df.filter(pl.col("rok").is_between(1949,1987)).select(pl.col("100_7")).to_series().to_list())
behem_padesatek = set(df.filter(pl.col("rok").is_between(1949,1959)).select(pl.col("100_7")).to_series().to_list())
pred_normalizaci = set(df.filter(pl.col("rok").is_between(1966,1970)).select(pl.col("100_7")).to_series().to_list())
po_normalizaci = set(df.filter(pl.col("rok").is_between(1990,1994)).select(pl.col("100_7")).to_series().to_list())
behem_normalizace = set(df.filter(pl.col("rok").is_between(1972,1988)).select(pl.col("100_7")).to_series().to_list())

znormalizovani = pred_normalizaci.intersection(po_normalizaci).difference(behem_normalizace)
zunorovani = pred_unorem.intersection(po_revoluci).difference(behem_socialismu)

In [28]:
len(znormalizovani)

508

In [29]:
len(df.filter(pl.col("100_7").is_in(set(cesi).intersection(znormalizovani))).unique(subset=['100_7']))

397

In [30]:
kolik = 23
znormalizovani_top_domaci = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
znormalizovani_top_domaci_razeni = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()
znormalizovani_top_zahranicni = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
znormalizovani_top_zahranicni_razeni = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()
znormalizovani_top_domaci_razeni

['Josef Škvorecký',
 'Ivan Klíma',
 'Ivan Sviták',
 'Václav Havel',
 'Anna Sedlmayerová',
 'Karel Kaplan',
 'Vladimír Škutina',
 'Jan Vladislav',
 'Jan Procházka',
 'Václav Černý',
 'Karel Šiktanc',
 'Jan Patočka',
 'Eva Kantůrková',
 'Milan Kundera',
 'Václav Renč',
 'Ivan Diviš',
 'Arnošt Lustig',
 'Adolf Melezinek',
 'Ludvík Vaculík',
 'Karel Pecka',
 'František Kautman',
 'Zdeněk Kalista',
 'Václav Klaus']

In [31]:
# kolik = 20
# znormalizovani_top_domaci = df.filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
# znormalizovani_top_domaci_razeni = df.filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()
# znormalizovani_top_zahranicni = df.filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
# znormalizovani_top_zahranicni_razeni = df.filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()

Načteme data vygenerovaná sešitem 174_zahranicni_vydani.ipynb.

In [33]:
horni_hranice = 2015

In [34]:
dografu2 = pl.read_parquet(os.path.join('data','zahranicni_knihy_10_zakazanych_autoru.parquet')).select(pl.col(['jmeno','titul','rok'])).with_columns(pl.lit('zahranici').alias('kde')).filter(pl.col("rok").is_between(1930,horni_hranice)).filter(pl.col("jmeno").is_in(znormalizovani_top_domaci_razeni))

In [35]:
dografu1 = df.filter(pl.col("rok").is_between(1930,horni_hranice)).filter(pl.col("100_7").is_in(znormalizovani_top_domaci)).select(pl.col(['jmeno','245_a','rok'])).rename({'245_a':'titul'}).with_columns(pl.lit('domaci').alias('kde'))

In [36]:
dografu = pl.concat([dografu1, dografu2]).filter(pl.col("jmeno").is_in(znormalizovani_top_domaci_razeni))

dografu = dografu.join(
    df.filter(
        pl.col("rok").is_between(1930,horni_hranice)
    ).filter(
        pl.col("100_7").is_in(znormalizovani_top_domaci)
    ).group_by('jmeno').agg(pl.col('umrti').max()), on='jmeno',how='left'
)

dografu = dografu.with_columns(
    pl.when(pl.col('umrti') > horni_hranice).then(None).otherwise(pl.col('umrti')).alias("umrti")
).with_columns(
    pl.col('umrti').map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime)
)

In [37]:
dografu

jmeno,titul,rok,kde,umrti
str,str,i64,str,datetime[μs]
"""Anna Sedlmayerová""","""Od večera do rána""",1969,"""domaci""",1995-01-01 00:00:00
"""Eva Kantůrková""","""Nad Pamětmi Václava Černého""",1995,"""domaci""",null
"""Karel Pecka""","""Dlouhý koktejl""",1991,"""domaci""",1997-01-01 00:00:00
"""Arnošt Lustig""","""Krásné zelené oči""",2007,"""domaci""",2011-01-01 00:00:00
"""Václav Renč""","""Jitření""",1933,"""domaci""",1973-01-01 00:00:00
"""Arnošt Lustig""","""Modlitba pro Kateřinu Horovitz…",2005,"""domaci""",2011-01-01 00:00:00
"""Karel Šiktanc""","""Pochodeň jara""",1954,"""domaci""",null
"""Milan Kundera""","""Nesmrtelnost""",1993,"""domaci""",null
"""Ivan Sviták""","""Sametová restaurace""",1993,"""domaci""",1994-01-01 00:00:00


In [38]:
# First create the base chart parameters
base = alt.Chart(alt_friendly(dografu.filter(pl.col("rok").is_between(1945,horni_hranice))), 
    title=alt.TitleParams(
        ["Normalizační průrva českou literaturou"], 
        subtitle=["Čeští autoři a autorky žijící v době srpnové okupace, kterým toho vyšlo nejvíc",
                  "v druhé půlce 60. let, první půlce 90. let a zároveň nic mezi lety 1973 a 1987.",
                  "Červené tečky jsou knihy vydané na českém území, modré vydané v zahraničí,",
                  "černé čárky jsou roky úmrtí a barevný pruh doba mezi okupací a revolucí."]
    ), width=kredity['sirka'] / 1.25)

# Create the yellow dots layer (foreign publications)
foreign = base.mark_circle(opacity=1).encode(
    x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    y=alt.Y("jmeno:N", sort=znormalizovani_top_domaci_razeni, title=None, 
            axis=alt.Axis(orient='left', domainOpacity=0, tickColor='white')),
    yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
    size=alt.value(6),
    color=alt.value('#81A9D5')
).transform_filter(
    alt.datum.kde == 'zahranici'
).transform_calculate(
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)

# Create the domestic publications layer
domestic = base.mark_circle(opacity=1).encode(
    x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    y=alt.Y("jmeno:N", sort=znormalizovani_top_domaci_razeni, title=None, 
            axis=alt.Axis(orient='left', domainOpacity=0, tickColor='white')),
    yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
    size=alt.value(9),
    color=alt.Color('domaci:N', 
                   scale=alt.Scale(range=['#D6534B']), 
                   sort=znormalizovani_top_domaci_razeni,
                   legend=None)
).transform_filter(
    alt.datum.kde == 'domaci'
).transform_calculate(
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)

kdy_umreli = base.mark_tick(
    color='black',  # optional: you can specify color
    thickness=1.75,
    height=9
).encode(
    x=alt.X('umrti:T', title=None),
    y=alt.Y("jmeno:N", sort=znormalizovani_top_domaci_razeni, title=None, 
            axis=alt.Axis(orient='left', domainOpacity=0, tickColor='white'))
)

okupace_cara = alt.Chart(alt.Data(values=[{'rok': '1968-08-21'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

revoluce_cara = alt.Chart(alt.Data(values=[{'rok': '1989-11-17'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

okupace_text = alt.Chart(alt.Data(values=[{'rok': '1968-08-21', 'y': 0.2}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    fontSize=8,
    dy=2,# Slight horizontal offset from the line
    dx=-2,
    text=['sametová','revoluce'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)

hranice_normalizace = pd.DataFrame([{"start": "1968-08-21", "end": "1989-11-17", "event": "normalizace"}])

rect = alt.Chart(hranice_normalizace).mark_rect(color='#ecf2f8').encode(
    x="start:T",
    x2="end:T"
)

# Layer the charts with foreign (yellow) on bottom and domestic on top
norm_dom = alt.layer(rect, kdy_umreli, foreign, domestic).configure_view(stroke='transparent').resolve_scale(color='independent',x="shared").configure_axisX(grid=False) # .configure_axisY(grid=True)

norm_dom

alt.LayerChart(...)

In [39]:
me_to_neurazi(norm_dom, kredity=kredity['default'], soubor="04_normalizace")

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/04_normalizace.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/04_normalizace.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [40]:
kredity['default']

'data: Česká národní bibliografie | vizualizace: iROZHLAS.cz | 2025'

In [41]:
znormalizovani_top_zahranicni_razeni

['Erich von Däniken',
 'Ladislav Mňačko',
 'Peter Ferdinand Drucker',
 'Erich Fromm',
 'Simone de Beauvoir',
 'Jean-Paul Sartre',
 'Rochdi Goulli',
 'Max Frisch',
 'Jean Genet',
 'Richmal Crompton',
 'Clifford D. Simak',
 'Peter Weiss',
 'Ladislav Grosman',
 'Eugène Ionesco',
 'Poul Anderson',
 'Marguerite Duras',
 'Günter Grass',
 'Nathalie Sarraute',
 'John Knittel',
 'André Pieyre de Mandiargues',
 'Wolfgang Ott',
 'Gertrud von Le Fort',
 'Cornelius Ryan']

## Stopky

In [43]:
nechceme = ['jn19990210621']
df = df.filter(~pl.col('100_7').is_in(nechceme))

In [124]:
df = df.with_columns(pl.col("jmeno").replace({"Ladislav Riedl Německobrodský": "L. R. Německobrodský"}))
df = df.with_columns(pl.col("jmeno").replace({"Il‘ja Grigor‘jevič Èrenburg": "Ilja Erenburg"}))
df = df.with_columns(pl.col("jmeno").replace({"Vasilij Ivanovič Nemirovič-Dančenko": "V. I. Nemirovič-Dančenko"}))
df = df.with_columns(pl.col("jmeno").replace({"Albert Vojtěch Velflík": "A. V. Velfík"}))
df = df.with_columns(pl.col("jmeno").replace({"Leonid Maksimovič Leonov": "L. M. Leonov"}))

In [125]:
def nevysli_po(rok, obdobi, nasobek):
    vysli_pred = df.filter(pl.col("rok").is_between(rok-obdobi,rok)).group_by('100_7').len().filter(pl.col('len') > 3).select(pl.col("100_7")).to_series().to_list()
    vysli_po = set(df.filter(pl.col("rok") > rok).select(pl.col("100_7")).to_series().to_list())
    ukonceni = [v for v in vysli_pred if v not in vysli_po]
    print(f"Filtru vyhovuje {len(ukonceni)} lidí.")
    return df.filter(pl.col("100_7").is_in(ukonceni)).filter(pl.col("rok").is_between(rok-(obdobi * nasobek),rok)).group_by(["jmeno","100_7"]).len().sort(by="len",descending=True).head(100)

In [126]:
sedesatosm = nevysli_po(1968, 2, 40).head(3)
sedesatosm

Filtru vyhovuje 19 lidí.


jmeno,100_7,len
str,str,u32
"""Antonín Václavovič""","""jk01141051""",23
"""František Vodička""","""jk01150244""",13
"""Miluše Nohejlová""","""jx20040805028""",11


In [127]:
tricetosm = nevysli_po(1938, 2, 40).head(3)
tricetosm

Filtru vyhovuje 48 lidí.


jmeno,100_7,len
str,str,u32
"""Augustin Matolín""","""jk01080750""",95
"""Nemirovič-Dančenko""","""jn20000604157""",66
"""Rudolf Vlasák""","""jk01150058""",58


In [128]:
ctyricetosm = nevysli_po(1948, 4, 40).head(3)
ctyricetosm

Filtru vyhovuje 196 lidí.


jmeno,100_7,len
str,str,u32
"""Quido Maria Vyskočil""","""jk01151235""",151
"""František Pulec""","""jk01101720""",75
"""Vláďa Zíka""","""jk01152620""",60


In [129]:
osmnact = nevysli_po(1918, 4, 40).head(3)
osmnact

Filtru vyhovuje 15 lidí.


jmeno,100_7,len
str,str,u32
"""Josef Flekáček""","""jk01031426""",24
"""Antonín Rosam""","""jk01102691""",24
"""A. V. Velfík""","""jk01141974""",21


In [130]:
ctyricetosm = nevysli_po(1948, 4, 40).head(3)
ctyricetosm

Filtru vyhovuje 196 lidí.


jmeno,100_7,len
str,str,u32
"""Quido Maria Vyskočil""","""jk01151235""",151
"""František Pulec""","""jk01101720""",75
"""Vláďa Zíka""","""jk01152620""",60


In [131]:
nevysli_po(1968, 4, 40).head(3)

Filtru vyhovuje 69 lidí.


jmeno,100_7,len
str,str,u32
"""Karel Andrlík""","""jk01010297""",74
"""Antonín Beneš""","""jk01011562""",24
"""Antonín Václavovič""","""jk01141051""",23


In [132]:
tricetdevet = nevysli_po(1939, 4, 40).head(3)
tricetdevet

Filtru vyhovuje 127 lidí.


jmeno,100_7,len
str,str,u32
"""Augustin Matolín""","""jk01080750""",95
"""Nemirovič-Dančenko""","""jn20000604157""",66
"""Rudolf Vlasák""","""jk01150058""",58


In [133]:
aut.sample(10)

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001
list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str
null,null,null,"[""1993""]","[""jn20031216026""]","[""Carvallo, Gastón,""]","[""-1993""]","[""1""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Venezuelský sociolog.""]",null,""" nz a22 n 4500""","""jn20031216026"""
null,null,"[""1926""]","[""1994""]","[""ola2003193705""]","[""Šášky, Ladislav,""]","[""1926-1994""]","[""1""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Slovenský historik, pedagog a muzejní pracovník. Zaměřen na církevní dějiny a dějiny umění.""]",null,""" nz a22 n 4500""","""ola2003193705"""
null,null,"[""1896""]","[""1952""]","[""jn19981002205""]","[""Tey, Josephine,""]","[""asi 1896-1952""]","[""1""]",null,null,null,null,null,null,null,null,null,null,"[""Mackintosh, Elizabeth,""]","[""asi 1896-1952""]","[""Skutečné jméno:""]","[""1""]",null,null,null,null,"[""Daviot, Gordon,""]",null,"[""1""]",null,"[""Britská spisovatelka.""]",null,""" cz a22 n 4500""","""jn19981002205"""
null,null,null,null,null,null,null,null,"[""Schweizerische Gesellschaft für Parodontologie""]",null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Société suisse de parodontologie"", ""Società svizzera di parodontologia"", … ""SSP""]",null,null,null,null,null,null,null,null,"[""https://www.parodontologie.ch/de/home""]",""" cz a22 n 4500""","""xx0061085"""
null,null,"[""1976""]",null,"[""jo2012729192""]","[""Toflová, Natália,""]","[""1976-""]","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,"[""Gardošová, Natália,""]","[""1976-""]",null,"[""1""]",null,null,null,null,null,null,null,null,"[""Narozena 6. 2. 1976. Mgr., lektorka a autorka publikací z oblasti dalšího vzdělávání pedagogických pracovníků, ředitelka mateřské školy.""]",null,""" nz a22 n 4500""","""jo2012729192"""
null,null,null,null,"[""jo20191024216""]","[""Askola, Heli""]",null,"[""1""]",null,null,null,"[""Finsko""]",null,"[""lidská práva"", ""migrace obyvatelstva"", ""migrační politika""]",null,"[""vysokoškolské učitelky""]","[""žena""]","[""eng""]",null,null,null,null,null,null,null,null,null,null,null,null,"[""Finská vysokoškolská pedagožka, zaměřuje se na lidská práva, migrační právo a politiku, management mezinárodní migrace, právo EU a rovnost pohlaví.""]",null,""" nz a22 n 4500""","""jo20191024216"""
null,null,"[""1987""]",null,"[""xx0314261""]","[""Martincová, Táňa,""]","[""1987-""]","[""1""]",null,null,null,"[""Česko""]","[""Praha, Česko""]","[""výtvarné umění"", ""grafika"", … ""literatura""]",null,"[""výtvarnice"", ""grafičky"", … ""spisovatelky""]","[""žena""]","[""cze""]","[""Sekerková, Taťána,"", ""Martincová, Taťána,""]","[""1987-"", ""1987-""]","[""Jméno získané sňatkem:"", ""Rodné jméno:""]","[""1"", ""1""]",null,null,null,null,null,null,null,null,"[""Narozena 3. 12. 1987. Výtvarnice, grafička a ilustrátorka, designérka, autorka cestovních her a autorských knih.""]",null,""" cz a22 n 4500""","""xx0314261"""
null,null,"[""1939""]",null,"[""jn20030409009""]","[""Winter, Hans-Gerhard,""]","[""1939-""]","[""1""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Literární vědec.""]",null,""" nz a22 n 4500""","""jn20030409009"""
null,null,"[""1897""]","[""1963""]","[""xx0173145""]","[""Brown, Rowland,""]","[""1897-1963""]","[""1""]",null,null,null,"[""Spojené státy americké""]",null,"[""film"", ""telev

In [134]:
osmdesatdevet = nevysli_po(1989, 4, 40).head(10)
osmdesatdevet

Filtru vyhovuje 228 lidí.


jmeno,100_7,len
str,str,u32
"""Antonín Zápotocký""","""jk01152141""",99
"""Ilja Erenburg""","""jn19990210194""",74
"""Josef Doubrava""","""jk01022875""",65
"""Josef Hrabák""","""jk01042553""",65
"""František Egermayer""","""jk01030317""",62
"""Marta Chytilová""","""jk01050494""",62
"""František Buriánek""","""jk01020403""",61
"""Donát Šajner""","""jk01122408""",59
"""Zdeněk Pluhař""","""jk01093515""",55


In [135]:
def lide(frejm):
    kody = frejm.select(pl.col('100_7')).to_series().to_list()
    for k in kody:
        clovek = aut.filter(pl.col('001') == k)
        print(clovek.select(pl.col('100_a')).to_series().to_list()[0])
        print(clovek.select(pl.col('678_a')).to_series().to_list()[0])
        print('***')

In [136]:
lide(tricetdevet)

['Matolín, Augustin,']
['Narozen 8.5.1869 v Brně, zemřel 21.2.1950 v Praze. Učitel, učebnice matematiky.']
***
['Nemirovič-Dančenko, Vasilij Ivanovič,']
['Ruský spisovatel, básník, prozaik, cestovatel a žurnalista, autor cestopisů, próz s válečnou tematikou a memoárové literatury, po roce 1921 v emigraci (nejdříve v Německu, později v Československu).']
***
['Vlasák, Rudolf,']
['Narozen 18.6.1888 v Praze, zemřel 4.6.1938 v Kolíně. Telegrafní zřízenec, legionářský spisovatel, prozaik, knihy pro děti, frašky a hry.']
***


In [137]:
lide(osmdesatdevet)

['Zápotocký, Antonín,']
['Narozen 19.12.1884 v Zákolanech u Kralup, zemřel 13.11.1957 v Praze. Komunistický funkcionář, prezident republiky, ekonomické publikace, próza.']
***
['Èrenburg, Il‘ja Grigor‘jevič,']
['Ruský spisovatel a básník, překladatel z francouzského a španělského jazyka, publicista.']
***
['Doubrava, Josef,']
['Narozen 5.7.1918 v Obděnicích u Votic, zemřel 1986. PaedDr., zeměpisná literatura.']
***
['Hrabák, Josef,']
['Narozen 3.12.1912 v Brně, zemřel 6.8.1987 tamtéž. PhDr., DrSc., profesor katedry českého a slovenského jazyka a literatury, literární vědec zabývající se dějinami české literatury, literární teorií, literárněvědnou metodologií, komparatistikou, kritikou současné produkce a ediční teorií a praxí.']
***
['Egermayer, František,']
['Narozen 15.8.1913 ve Zbůchu u Plzně, zemřel 13.10.1989 v Praze. RNDr., DrSc., profesor statistiky, matematik, technik, ekonom, členem mezinárodních organizací pro řízení jakosti, práce z oborů.']
***
['Chytilová, Marta,']
['Naroz

In [138]:
#fertig = pl.concat([osmnact, ctyricetosm, osmdesatdevet])

fertig_18_kdo = osmnact.select(pl.col('100_7')).to_series().to_list()
fertig_18_razeni = osmnact.select(pl.col('jmeno')).to_series().to_list()
fertig_18_do_grafu = df.filter(pl.col('100_7').is_in(fertig_18_kdo))

fertig_39_kdo = tricetdevet.select(pl.col('100_7')).to_series().to_list()
fertig_39_razeni = tricetdevet.select(pl.col('jmeno')).to_series().to_list()
fertig_39_do_grafu = df.filter(pl.col('100_7').is_in(fertig_39_kdo))

fertig_48_kdo = ctyricetosm.select(pl.col('100_7')).to_series().to_list()
fertig_48_razeni = ctyricetosm.select(pl.col('jmeno')).to_series().to_list()
fertig_48_do_grafu = df.filter(pl.col('100_7').is_in(fertig_48_kdo))

fertig_89_kdo = osmdesatdevet.select(pl.col('100_7')).to_series().to_list()
fertig_89_razeni = osmdesatdevet.select(pl.col('jmeno')).to_series().to_list()
fertig_89_do_grafu = df.filter(pl.col('100_7').is_in(fertig_89_kdo))

In [139]:
fertig_48_razeni

['Quido Maria Vyskočil', 'František Pulec', 'Vláďa Zíka']

In [171]:
fertig_18_graf = alt.Chart(alt_friendly(fertig_18_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(
    ["Nejvydávanější čeští i zahraniční autoři, kteří už nikdy nevyšli","po vyhlášení Československa…"], dx=-22,
)).mark_circle(size=9, opacity=1) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_18_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B']), 
                            sort=fertig_18_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

fertig_39_graf = alt.Chart(alt_friendly(fertig_39_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(["…po nacistické okupaci…"], 
)).mark_circle(size=9, opacity=1) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_39_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B']), 
                            sort=fertig_39_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

fertig_48_graf = alt.Chart(alt_friendly(fertig_48_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(["…po únorovém převratu…"], 
)).mark_circle(size=9, opacity=1) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_48_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B']), 
                            sort=fertig_48_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

fertig_89_graf = alt.Chart(alt_friendly(fertig_89_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(
    ["…a po revoluci"], dx=-4
)).mark_circle(size=9, opacity=1) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_89_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B']), 
                            sort=fertig_89_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

zmenseni = 1.25

seznamy_koncicich = alt.vconcat(
    fertig_18_graf.properties(
    width=kredity['sirka'] / zmenseni), 
    fertig_39_graf.properties(
    width=kredity['sirka'] / zmenseni), 
    fertig_48_graf.properties(
    width=kredity['sirka'] / zmenseni), 
    fertig_89_graf.properties(
    width=kredity['sirka'] / zmenseni)
).resolve_axis(x="shared").configure_view(stroke='transparent')

seznamy_koncicich

alt.VConcatChart(...)

In [173]:
me_to_neurazi(seznamy_koncicich, soubor="04_seznamy_koncicich", kredity=kredity['default'])

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/04_seznamy_koncicich.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/04_seznamy_koncicich.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [142]:
df.filter(pl.col("jmeno") == "Jarmila Šteflíčková")

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,narozeni,umrti,gender,vek,jmeno
str,str,str,list[str],str,str,list[str],str,str,str,str,str,list[str],list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,i64,i64,str,i64,str
"""1""","""Šteflíčková, Jarmila,""","""jk01131190""","[""aut""]","""1926-2001""",null,null,null,null,"""ck8806196""",""" nam a22 4500""","""881011s1988 xr u0…","[""(Brož.)""]",null,null,null,null,null,null,null,null,"""1""","""0""","""České dějiny v krásné próze""","""Soupis knih /""","""Zprac. Jarmila Šteflíčková ; G…",null,null,null,null,null,"[""76 s. ;""]",null,"[""20 cm""]",null,null,null,…,"[""czenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1988,76,1926,2001,"""f""",62,"""Jarmila Šteflíčková"""
"""1""","""Šteflíčková, Jarmila,""","""jk01131190""","[""com""]","""1926-2001""",null,null,null,null,"""bk196401903""",""" nam a22 1 4500""","""971016s1964 xr …",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Co recitovat ze sovětské poezi…","""Výběr sov. veršů a básnických …","""Sest. Jarmila Šteflíčková ; [Z…",null,null,null,null,null,"[""61, [1] s. ;""]",null,"[""8°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1964,61,1926,2001,"""f""",38,"""Jarmila Šteflíčková"""
"""1""","""Šteflíčková, Jarmila,""","""jk01131190""","[""com""]","""1926-2001""",null,null,null,null,"""bk196801541""",""" nam a22 1 4500""","""970611s1968 xr …",null,null,null,null,null,null,null,null,null,"""1""","""0""","""V branách nového věku""","""Výběr lit. o údobí od poloviny…","""Sest. Jarmila Šteflíčková a Ma…",null,null,null,null,null,"[""43, [1] s. ;""]",null,"[""8°""]",null,null,null,…,null,null,null,null,"[""1""]","[""Tučková, Marie,""]","[""com""]","[""1914-""]","[""mzk2006355704""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1968,43,1926,2001,"""f""",42,"""Jarmila Šteflíčková"""
"""1""","""Šteflíčková, Jarmila,""","""jk01131190""","[""com""]","""1926-2001""",null,null,null,null,"""bk196103067""",""" nam a22 1 4500""","""980423s1961 xr …",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Co číst o životě probouzejícíc…","""Výběrový seznam beletrie /""","""Sest. Jarmila Šteflíčková""",null,null,null,null,null,"[""40 s. ;""]",null,"[""8°""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1961,40,1926,2001,"""f""",35,"""Jarmila Šteflíčková"""
"""1""","""Šteflíčková, Jarmila,""","""jk01131190""","[""aut""]","""1926-2001""",null,null,null,null,"""ck8702849""",""" nam a22 4500""","""870713s1986 xr a u0…","[""(Brož.)""]",null,null,null,null,null,null,null,null,"""1""","""0""","""Péče o životní prostředí""","""Soupis knih /""","""Zprac. Jarmila Šteflíčková ; I…",null,null,null,null,null,"[""90 s. :""]","[""obr. ;""]","[""20 cm""]",null,null,null,…,"[""czenas""]",null,null,null,"[""1"", ""1"", ""1""]","[""Drahokoupil, Jaros

## Lze najít rozdíl napsáno-vydáno?

In [144]:
df.filter(pl.col("245_a").str.contains("Zbabělci"))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,narozeni,umrti,gender,vek,jmeno
str,str,str,list[str],str,str,list[str],str,str,str,str,str,list[str],list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,i64,i64,str,i64,str
"""1""","""Škvorecký, Josef,""","""jk01130413""","[""aut""]","""1924-2012""",null,null,null,null,"""nkc20051533683""",""" nam a22 a 4500""","""050505s2005 xr g 0…","[""(váz.)""]",null,"[""80-86938-05-0""]",null,null,null,null,null,null,"""1""","""0""","""Zbabělci""",null,"""Josef Škvorecký""",null,null,null,null,null,"[""379 s. ;""]",null,"[""30 cm""]",null,null,null,…,"[""czenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2005,379,1924,2012,"""m""",81,"""Josef Škvorecký"""
"""1""","""Škvorecký, Josef,""","""jk01130413""","[""aut""]","""1924-2012""",null,null,null,null,"""nkc20213369425""",""" nam a22 i 4500""","""211104s2020 xr g 0…","[""(Nadační fond Česká knižnice ;"", ""vázáno)"", … ""vázáno)""]",null,"[""978-80-88183-24-2"", ""978-80-7658-001-5"", … ""978-80-275-0396-4""]",null,null,null,null,null,null,"""1""","""0""","""Zbabělci""",null,"""Josef Škvorecký""",null,null,null,null,null,"[""449 stran ;""]",null,"[""21 cm""]",null,null,null,…,"[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2020,449,1924,2012,"""m""",96,"""Josef Škvorecký"""
"""1""","""Blažíček, Přemysl,""","""jk01012340""","[""aut""]","""1932-2002""",null,null,null,null,"""np9315849""",""" nam a22 4500""","""940120s1992 xr g 0…","[""(brož.)""]",null,"[""80-85241-05-6""]",null,null,null,null,null,null,"""1""","""0""","""Škvoreckého Zbabělci""",null,"""Přemysl Blažíček""",null,null,null,null,null,"[""68 s. ;""]",null,"[""20 cm""]",null,null,null,…,"[""czenas"", ""eczenas""]",null,null,null,"[""1""]","[""Škvorecký, Josef,""]",null,"[""1924-2012.""]","[""aun2007414779""]","[""Zbabělci""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1992,68,1932,2002,"""m""",60,"""Přemysl Blažíček"""
"""1""","""Blažíček, Přemysl,""","""jk01012340""","[""aut""]","""1932-2002""",null,null,null,null,"""nkc20152662766""",""" nam a22 a 4500""","""150202s2014 xr e 0…","[""(brož.)""]",null,"[""978-80-7474-103-6""]",null,null,null,null,null,null,"""1""","""0""","""Knihy o epice: Naši, Švejk, Zb…",null,"""Přemysl Blažíček ; uspořádal, …",null,null,null,null,null,"[""583 s. ;""]",null,"[""19 cm""]",null,null,null,…,"[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"[""1""]","[""Špirit, Michael,""]","[""edt""]","[""1965-""]","[""jn20001103589""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2014,583,1932,2002,"""m""",82,"""Přemysl Blažíček"""
"""1""","""Zachrdla, Jakub,""","""xx0268749""","[""aut""]","""1987-""",null,null,null,null,"""nkc20233506813""",""" nam a22 i 4500""","""230313s2023 xr g 0…","[""(brožováno) :""]","[""Kč 449,00""]","[""978-80-271-3816-6""]",null,null,null,null,null,null,"""1""","""0""","""Zbabělci a vrazi""","""v temných zá